In [19]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Concatenate, Dropout, BatchNormalization
from keras.utils.vis_utils import plot_model
 
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import scipy as sp
import seaborn as sns

import numpy as np

import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tabulate import tabulate

import time

from multiprocessing import Pool
from joblib import Parallel, delayed
import concurrent.futures

import pickle
import os

data_path  = './Dataset_Cleaned/'
model_path = './Model_Saved/'
newdata_path = './v6_A-B-C-D-E_Sputtering_ML_master/v6_A-B-C-D-E_Sputtering_ML_before/'

# ----- BO-related -----
notebook_fname = "NN_full_RepeatedKFold_v3_BO_2"
num_str        = notebook_fname.split("_")[-1]

model_path_bo = f'{model_path}/{notebook_fname}/'

if not os.path.exists(model_path_bo):
    os.makedirs(model_path_bo)
    print(f"Folder '{model_path_bo}' created.")
else:
    print(f"Folder '{model_path_bo}' model path found.")
# ----- BO-related -----

tf.keras.backend.clear_session()

Folder './Model_Saved//NN_full_RepeatedKFold_v3_BO_2/' model path found.


#### load scalers

In [20]:
# Load the scalers from the file
with open(data_path + 'scalers.pkl', 'rb') as f:
    (scaler_compo, 
     scaler_H_specific, 
     scaler_C_specific, 
     scaler_H_output, 
     scaler_C_output) = pickle.load(f)

#### coordinates for PVD alloy representation

In [21]:
# load the wafer-coordinates
df_PVD_x_y = pd.read_excel(data_path + 'PVD_x_y.xlsx')
coord_x = df_PVD_x_y["x"].to_numpy(dtype=float)
coord_y = df_PVD_x_y["y"].to_numpy(dtype=float)
index_PVD_x_y = df_PVD_x_y.index.values+1
 

#### define `read_new_data` to load the inputs for making predictions

In [22]:
def read_new_data(df_new_at, df_new_wt, vars_ele): 
       
       # ----- import the new dataset ----- 
       vars_ele  = vars_ele

       # ----- get the `df_compo_new` ------
       df_compo = pd.DataFrame(columns=['Fe', 'Cr', 'Ni', 'Mo', 'W', 'N', 'Nb', 'C', 'Si', 'Mn',
              'Cu', 'Al', 'V', 'Ta', 'Ti', 'Co', 'Mg', 'Y', 'Zr', 'Hf'])

       df_compo_new = pd.concat([df_compo, df_new_wt[vars_ele]], axis=0, ignore_index=True)
       df_compo_new = df_compo_new.fillna(0) # fill in the nan by zero

       compo_new    = np.asarray(df_compo_new.values)

       # ----- get the `df_H_specific_new` ----- 
       df_H_specific_new = df_new_at[['delta_a', 'Hmix', 'VEC', 'sigma_bulk_modulus']]

       H_specific_new = np.asarray(df_H_specific_new.values)


       # ----- get the `df_C_specifi_new` ----- 
       C_testing = np.array([25, 0.5, 7, 0.215]) # TestTemperature_C, ChlorideIonConcentration, pH, ScanRate_mVs
       C_specific_new = np.ones((len(df_compo_new. index), len(C_testing))) * C_testing # corrosion testing condition array
       # df_C_specific_new = pd.DataFrame(C_specific_new, columns=['TestTemperature_C', 'ChlorideIonConcentration', 'pH', 'ScanRate_mVs'])

       return compo_new, H_specific_new, C_specific_new


#### define `predict_bootstrap_NNH_NNC` for predictions from NNH and NNC

In [23]:
def predict_bootstrap(model_path, model_name,
                      X1_list, Y1_list, 
                      k_folds, n_CVrepeats, mc_repeat,
                      scaler_compo, scaler_H_specific, scaler_output):
    
    H1_pred_X1_list    = []
    H1_pred_X1_mc_mean = []
    H1_pred_X1_mc_std  = []
    
    def predict_one_model(i):

        # loading saved models
        NNH_model_loaded_temp = keras.models.load_model(os.path.join(model_path, model_name.format(i+1)))
        
        # make prediction
        def predict_one_sample():
            return scaler_output.inverse_transform(
                NNH_model_loaded_temp.predict([scaler_compo.transform(X1_list[i]), 
                                                   scaler_H_specific.transform(Y1_list[i])], 
                                                   verbose = 0)
                                                  )
            
        H1_pred_X1_mc_stack_temp = tf.map_fn(lambda _: predict_one_sample(),
                                             tf.range(mc_repeat),
                                             dtype=tf.float32,
                                             parallel_iterations=mc_repeat)
        
        H1_pred_X1_mc_mean_temp = np.mean(H1_pred_X1_mc_stack_temp, axis = 0).reshape((-1,))
        H1_pred_X1_mc_std_temp  = np.std(H1_pred_X1_mc_stack_temp,  axis = 0).reshape((-1,))
        
        return H1_pred_X1_mc_stack_temp, H1_pred_X1_mc_mean_temp, H1_pred_X1_mc_std_temp
    
    results = Parallel(n_jobs=-1)(delayed(predict_one_model)(i) for i in range(k_folds * n_CVrepeats))
    
    # clear TensorFlow session
    tf.keras.backend.clear_session()

    for mc_stack, mean, std in results:
        H1_pred_X1_list.append(mc_stack)
        H1_pred_X1_mc_mean.append(mean)
        H1_pred_X1_mc_std.append(std)

    return H1_pred_X1_list, H1_pred_X1_mc_mean, H1_pred_X1_mc_std

In [24]:
def predict_bootstrap_NNH_NNC(model_path_bo, NNH_model_name, NNC_model_name,
                               compo_new, H_specific_new, C_specific_new,
                               scaler_compo, scaler_H_specific, scaler_C_specific,
                               scaler_H_output, scaler_C_output,
                               k_folds, n_CVrepeats, mc_repeat):
    
    # Repeat new data to match number of CV repetitions
    compo_new_list = [compo_new]*k_folds*n_CVrepeats
    H_specific_new_list = [H_specific_new]*k_folds*n_CVrepeats
    C_specific_new_list = [C_specific_new]*k_folds*n_CVrepeats

    # Predict using bootstrap resampling in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future1 = executor.submit(predict_bootstrap, model_path_bo, NNH_model_name, 
                                                 compo_new_list, H_specific_new_list, 
                                                 k_folds, n_CVrepeats, mc_repeat,
                                                 scaler_compo, scaler_H_specific, scaler_H_output)
        future2 = executor.submit(predict_bootstrap, model_path_bo, NNC_model_name, 
                                                 compo_new_list, C_specific_new_list, 
                                                 k_folds, n_CVrepeats, mc_repeat,
                                                 scaler_compo, scaler_C_specific, scaler_C_output)

    # Retrieve results from parallel predictions
    H1_new_pred_stack, H1_new_pred_mean, H1_new_pred_std = future1.result()
    C2_new_pred_stack, C2_new_pred_mean, C2_new_pred_std = future2.result()
    
    return H1_new_pred_stack, H1_new_pred_mean, H1_new_pred_std, C2_new_pred_stack, C2_new_pred_mean, C2_new_pred_std

#### define `plot_prediction_uncertainty` for prediction and uncertainty for each fold in bootstrap on wafer-coordinates

In [25]:
def plot_prediction_uncertainty(coord_x, coord_y, index_PVD_x_y, 
                                pred_mean, pred_std, pred_label, unc_label, 
                                title, vmin1, vmax1, vmin2, vmax2, model_path):
    
    fig, axs = plt.subplots(nrows=4, ncols=6, figsize=(18, 10), dpi=200)

    for i, (ax_pred, ax_unc) in enumerate(zip(axs[::2].flat, axs[1::2].flat)):
        # ----- subplot 1 prediction -----
        cax1 = ax_pred.scatter(coord_x, coord_y, c=pred_mean[i], s=400, marker='.', cmap='RdBu_r', vmin=vmin1, vmax=vmax1)
        ax_pred.set_xlabel('X')
        ax_pred.set_ylabel('Y')
        ax_pred.set_title(f'prediction {i+1}', fontsize=8)
        ax_pred.set_aspect('equal', 'box')
        for i_pvd, txt in enumerate(index_PVD_x_y):
            ax_pred.annotate(txt, (coord_x[i_pvd]-2, coord_y[i_pvd]-1), color="grey", alpha=1, fontsize=8)

        cbar1 = fig.colorbar(cax1, ax=ax_pred)
        cbar1.set_label(pred_label, size=8)
        cbar1.ax.tick_params(labelsize=8)

        # ----- subplot 2 prediction uncertainty -----
        cax2 = ax_unc.scatter(coord_x, coord_y, c=pred_std[i], s=400, marker='.', cmap='RdGy_r', vmin=vmin2, vmax=vmax2)
        ax_unc.set_xlabel('X')
        ax_unc.set_ylabel('Y')
        ax_unc.set_title(f'prediction uncertainty {i+1}', fontsize=8)
        ax_unc.set_aspect('equal', 'box')
        for i, txt in enumerate(index_PVD_x_y):
            ax_unc.annotate(txt, (coord_x[i]-3, coord_y[i]-1.5), color="grey", alpha=1, fontsize=8)

        cbar2 = fig.colorbar(cax2, ax=ax_unc)
        cbar2.set_label(unc_label, size=8)
        cbar2.ax.tick_params(labelsize=8)
        # -----------------------------------

    # adjust spacing and show plot
    plt.suptitle(title, fontsize=18)
    fig.tight_layout()
    plt.savefig(model_path + title + '.png', bbox_inches='tight')
    plt.show()


#### define `plot_prediction_uncertainty_AVG` for prediction and uncertainty for averaged bootstarp on wafer-coordinates

In [26]:
def plot_prediction_uncertainty_AVG(coord_x, coord_y, index_PVD_x_y, H1_new_pred_stack, C2_new_pred_stack, 
                                    title, KW_name, model_path_bo):
    
    # ----- concatenate and compute mean and std -----
    H1_new_pred_KFold_mean = np.mean(np.concatenate(H1_new_pred_stack, axis=0), axis=0).reshape(-1)
    H1_new_pred_KFold_std  = np.std(np.concatenate(H1_new_pred_stack, axis=0), axis=0).reshape(-1)

    C2_new_pred_KFold_mean = np.mean(np.concatenate(C2_new_pred_stack, axis=0), axis=0).reshape(-1)
    C2_new_pred_KFold_std  = np.std(np.concatenate(C2_new_pred_stack, axis=0), axis=0).reshape(-1)

    # ----- plot true vs prediction for all the RepeatedKFold + MC_dropout -----
    # Set font size for all text in figure
    plt.rcParams.update({'font.size': 8})

    # Create figure with 2 subplots
    fig, ax = plt.subplots(2, 2, figsize=(6, 5), dpi=200)

    # Define plot details
    plot_details = [(H1_new_pred_KFold_mean, H1_new_pred_KFold_std, 'Hardness', ''),    
                    (C2_new_pred_KFold_mean, C2_new_pred_KFold_std, 'Pitting potential', '(mV)')]

    # Loop through plot details and create subplots
    for i, (mean, std, name, unit) in enumerate(plot_details):
        row, col = i // 2, i % 2
        cmap1, cmap2 = plt.get_cmap('RdBu_r'), plt.get_cmap('RdGy_r')
        cax1 = ax[row, col].scatter(coord_x, coord_y, c=mean, s=400, marker='.', cmap=cmap1)
        cax2 = ax[row+1, col].scatter(coord_x, coord_y, c=std, s=400, marker='.', cmap=cmap2)
        ax[row, col].set_title(f'{name} ')
        ax[row+1, col].set_title(f'{name}  uncertainty')
        ax[row, col].set_aspect('equal')
        ax[row+1, col].set_aspect('equal')
        ax[row, col].set_xlabel('X')
        ax[row, col].set_ylabel('Y')
        ax[row+1, col].set_xlabel('X')
        ax[row+1, col].set_ylabel('Y')
        for i_pvd, txt in enumerate(index_PVD_x_y):
            ax[row, col].annotate(txt, (coord_x[i_pvd]-3, coord_y[i_pvd]-1.5), color="grey", alpha=1)
            ax[row+1, col].annotate(txt, (coord_x[i_pvd]-3, coord_y[i_pvd]-1.5), color="grey", alpha=1)
        cbar1, cbar2 = fig.colorbar(cax1, ax=ax[row, col]), fig.colorbar(cax2, ax=ax[row+1, col])
        cbar1.set_label(f'{name}  {unit}')
        cbar2.set_label(f'{name}  uncertainty {unit}')

    # adjust spacing and show plot
    plt.suptitle(title, fontsize=10)

    # Save and show figure
    fig.tight_layout()
    plt.savefig(model_path_bo + 'NN_full_RepeatedKFold_prediction_uncertainty_AVG_' + KW_name + '.png', bbox_inches='tight')
    plt.show()

#### make predictions for new alloys 

In [33]:
from itertools import combinations

# Define the set of elements for D and E
set_D_E = [{'Co'}, {'V'}, {'Mn'}, {'Mo'}, {'Cu'}, {'Nb'}, {'W'}, {'Ti'}, {'Al'}, {'Si'}, {'Ta'}]

# Generate all possible combinations of D and E
compo_D_E_list = list(combinations(set_D_E, 2))

compo_A_B_C_D_E_list       = []
compo_A_B_C_D_E_fname_list = []
for compo_D_E in compo_D_E_list:
    # Convert the current combination to a list and flatten it
    compo_D_E_list = [item for sublist in compo_D_E for item in sublist]
    
    # Define the current combination of A, B, C, D, E
    compo_A_B_C_D_E = ['Fe', 'Cr', 'Ni'] + compo_D_E_list
    compo_A_B_C_D_E_merge = '_'.join([*compo_A_B_C_D_E])

    compo_A_B_C_D_E_list.append(compo_A_B_C_D_E)
    compo_A_B_C_D_E_fname_list.append(compo_A_B_C_D_E_merge)

In [56]:
from tqdm import tqdm

for compo_A_B_C_D_E, compo_A_B_C_D_E_fname in tqdm(zip(compo_A_B_C_D_E_list, compo_A_B_C_D_E_fname_list), desc='Processing', total=len(compo_A_B_C_D_E_list)):
    print(compo_A_B_C_D_E)
    print(compo_A_B_C_D_E_fname)

    # compo_A_B_C_D_E       = compo_A_B_C_D_E_list[1]; 
    # compo_A_B_C_D_E_fname = compo_A_B_C_D_E_fname_list[1]; 

    df_new_at = pd.read_excel(newdata_path + 'v6_' + compo_A_B_C_D_E_fname + '_SSS_FCC_byCompo_at_pct.xlsx')
    df_new_wt = pd.read_excel(newdata_path + 'v6_' + compo_A_B_C_D_E_fname + '_SSS_FCC_byCompo_wt_pct.xlsx')
    vars_ele =  compo_A_B_C_D_E
        
    # ----- make predition -----
    k_folds     = 6
    n_CVrepeats = 2
    mc_repeat   = 10

    NNH_model_name = 'NNH_model_RepeatedKFold_{}.h5'
    NNC_model_name = 'NNC_model_RepeatedKFold_{}.h5'

    compo_new, H_specific_new, C_specific_new = read_new_data(df_new_at, df_new_wt, vars_ele)

    (H1_new_pred_stack, H1_new_pred_mean, H1_new_pred_std, 
    C2_new_pred_stack, C2_new_pred_mean, C2_new_pred_std) = predict_bootstrap_NNH_NNC(
                                                                model_path_bo, NNH_model_name, NNC_model_name,
                                                                compo_new, H_specific_new, C_specific_new,
                                                                scaler_compo, scaler_H_specific, scaler_C_specific, 
                                                                scaler_H_output, scaler_C_output,
                                                                k_folds, n_CVrepeats, mc_repeat)

    # ----- concatenate and compute mean and std + save to excel -----
    H1_new_pred_KFold_mean = np.mean(np.concatenate(H1_new_pred_stack, axis=0), axis=0).reshape(-1)
    H1_new_pred_KFold_std  = np.std(np.concatenate(H1_new_pred_stack, axis=0), axis=0).reshape(-1)
    C2_new_pred_KFold_mean = np.mean(np.concatenate(C2_new_pred_stack, axis=0), axis=0).reshape(-1)
    C2_new_pred_KFold_std  = np.std(np.concatenate(C2_new_pred_stack, axis=0), axis=0).reshape(-1)

    df_new_at['H1_new_pred_KFold_mean'] = H1_new_pred_KFold_mean
    df_new_at['H1_new_pred_KFold_std']  = H1_new_pred_KFold_std
    df_new_at['C2_new_pred_KFold_mean'] = C2_new_pred_KFold_mean
    df_new_at['C2_new_pred_KFold_std']  = C2_new_pred_KFold_std

    # display(df_new_at)
    df_new_at.to_excel(newdata_path + 'v6_' + compo_A_B_C_D_E_fname + '_SSS_FCC_byCompo_at_pct_ML.xlsx')


Processing:   0%|          | 0/55 [00:00<?, ?it/s]

['Fe', 'Cr', 'Ni', 'Co', 'V']
Fe_Cr_Ni_Co_V


Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7f24ac7f5820>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7fb14c30ab80>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7f4df005eb80>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7f5ac4448b80>: no matching AST found among candidates:

To silence thi

,Var1,Fe,Cr,Ni,Co,V,ShearModulus_eff,PoissonsRatio_eff,delta_prime_misfit,sigma_SSS,...,compo_A_B_C_D_E,FORMULA,delta_a,Hmix,VEC,sigma_bulk_modulus,H1_new_pred_KFold_mean,H1_new_pred_KFold_std,C2_new_pred_KFold_mean,C2_new_pred_KFold_std
0,0,33.0,3.0,5.0,12.0,47.0,63.740744,0.329844,0.092273,456.766541,...,Fe_Cr_Ni_Co_V,Fe33Cr3Ni5Co12V47,0.028152,-9.735600,6.750000,7.490661,222.351852,100.376816,261.414612,231.712891
1,1,39.0,5.0,5.0,11.0,40.0,66.837689,0.322980,0.091240,464.791066,...,Fe_Cr_Ni_Co_V,Fe39Cr5Ni5Co11V40,0.027861,-8.995600,6.910000,7.251896,237.455154,90.997993,262.986542,215.029984
2,2,44.0,7.0,4.0,10.0,34.0,69.598860,0.316791,0.088701,457.512234,...,Fe_Cr_Ni_Co_V,Fe44Cr7Ni4Co10V34,0.027185,-8.053056,7.010101,6.936681,257.467896,93.305237,281.539246,183.017761
3,3,49.0,11.0,4.0,9.0,27.0,73.592149,0.307933,0.084017,436.725429,...,Fe_Cr_Ni_Co_V,Fe49Cr11Ni4Co9V27,0.025549,-6.910800,7.140000,6.689544,288.463440,98.473129,341.054871,151.468628
4,4,53.0,14.0,3.0,7.0,22.0,76.515688,0.301265,0.079044,406.690501,...,Fe_Cr_Ni_Co_V,Fe53Cr14Ni3Co7V22,0.024064,-5.811244,7.181818,6.290262,271.782745,106.803040,404.398651,164.149994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8275,8275,6.0,32.0,50.0,9.0,3.0,87.945628,0.281502,0.055510,252.089626,...,Fe_Cr_Ni_Co_V,Fe6Cr32Ni50Co9V3,0.009584,-6.637600,8.360000,9.393615,165.298294,66.848938,520.209656,77.834602
8276,8276,6.0,25.0,52.0,13.0,4.0,85.001821,0.289050,0.057914,263.407680,...,Fe_Cr_Ni_Co_V,Fe6Cr25Ni52Co13V4,0.010964,-6.436800,8.550000,9.002222,140.323578,37.589668,418.387512,152.726364
8277,8277,5.0,19.0,53.0,18.0,5.0,82.445642,0.295652,0.060127,273.748984,...,Fe_Cr_Ni_Co_V,Fe5Cr19Ni53Co18V5,0.012130,-6.210800,8.710000,8.538735,122.341728,24.239143,256.332428,160.767502
8278,8278,4.0,13.0,53.0,23.0,6.0,79.942359,0.302087,0.064931,304.128710,...,Fe_Cr_Ni_Co_V,Fe4Cr13Ni53Co23V6,0.013217,-5.945108,8.858586,7.925271,114.139122,20.217979,167.726288,147.525543


Processing:   2%|▏         | 1/55 [00:14<13:13, 14.69s/it]

...

Processing:  98%|█████████▊| 54/55 [13:19<00:14, 14.96s/it]

['Fe', 'Cr', 'Ni', 'Si', 'Ta']
Fe_Cr_Ni_Si_Ta


Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7fc7a83ee820>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7fb1346cff70>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7f45f46b8c10>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function predict_bootstrap.<locals>.predict_one_model.<locals>.<lambda> at 0x7f7c5c4b8820>: no matching AST found among candidates:

To silence thi

,Var1,Fe,Cr,Ni,Si,Ta,ShearModulus_eff,PoissonsRatio_eff,delta_prime_misfit,sigma_SSS,...,compo_A_B_C_D_E,FORMULA,delta_a,Hmix,VEC,sigma_bulk_modulus,H1_new_pred_KFold_mean,H1_new_pred_KFold_std,C2_new_pred_KFold_mean,C2_new_pred_KFold_std
0,0,33.0,3.0,5.0,12.0,47.0,64.966643,0.335322,0.154332,1095.396810,...,Fe_Cr_Ni_Si_Ta,Fe33Cr3Ni5Si12Ta47,0.081320,-32.310800,6.150000,31.371803,777.461670,190.630142,204.997025,495.328125
1,1,39.0,5.0,5.0,11.0,40.0,66.750948,0.329984,0.159378,1157.692614,...,Fe_Cr_Ni_Si_Ta,Fe39Cr5Ni5Si11Ta40,0.079158,-30.100000,6.360000,29.841414,747.255981,184.449661,249.576797,481.935364
2,2,44.0,7.0,4.0,10.0,34.0,68.529438,0.324783,0.160516,1179.354805,...,Fe_Cr_Ni_Si_Ta,Fe44Cr7Ni4Si10Ta34,0.076399,-27.562902,6.505051,28.400189,712.797180,172.101089,254.274719,450.559662
3,3,49.0,11.0,4.0,9.0,27.0,71.225314,0.316913,0.157303,1159.409800,...,Fe_Cr_Ni_Si_Ta,Fe49Cr11Ni4Si9Ta27,0.071300,-24.176400,6.690000,26.415715,643.494202,151.102524,308.079285,414.151276
4,4,53.0,14.0,3.0,7.0,22.0,74.445010,0.309456,0.151576,1123.568850,...,Fe_Cr_Ni_Si_Ta,Fe53Cr14Ni3Si7Ta22,0.066031,-19.993470,6.828283,23.733290,551.012024,139.131607,357.876648,376.078522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8275,8275,6.0,32.0,50.0,9.0,3.0,78.246602,0.292457,0.094979,534.588988,...,Fe_Cr_Ni_Si_Ta,Fe6Cr32Ni50Si9Ta3,0.034050,-19.736800,7.910000,23.174124,522.155151,213.379562,574.254700,130.399841
8276,8276,6.0,25.0,52.0,13.0,4.0,71.734913,0.304431,0.107319,600.384539,...,Fe_Cr_Ni_Si_Ta,Fe6Cr25Ni52Si13Ta4,0.039976,-24.669200,7.900000,26.887916,680.379639,243.273239,530.531982,122.000465
8277,8277,5.0,19.0,53.0,18.0,5.0,65.081936,0.316426,0.119971,653.416442,...,Fe_Cr_Ni_Si_Ta,Fe5Cr19Ni53Si18Ta5,0.045668,-30.161200,7.810000,30.654690,855.412720,234.653488,381.994781,139.252655
8278,8278,4.0,13.0,53.0,23.0,6.0,58.816871,0.328312,0.135912,723.214547,...,Fe_Cr_Ni_Si_Ta,Fe4Cr13Ni53Si23Ta6,0.050912,-35.179267,7.696970,33.931577,982.748474,207.585266,260.400940,173.688812


Processing: 100%|██████████| 55/55 [13:34<00:00, 14.81s/it]


In [ ]:
# # ----- plotting -----
# # NNH predictions
# pred_label, unc_label = 'Hardness', 'Hardness uncertainty'
# title                 = 'NNH_RepeatedKFold_prediction_uncertainty_eachFold_' + KW_name
# vmin1, vmax1          = 100, 300
# vmin2, vmax2          = 25, 100
# plot_prediction_uncertainty(coord_x, coord_y, index_PVD_x_y, H1_new_pred_mean, H1_new_pred_std, 
#                             pred_label, unc_label, title, vmin1, vmax1, vmin2, vmax2, model_path_bo)

# # NNC predictions
# pred_label, unc_label = 'Pitting potential (mV)', 'Pitting potential uncertainty (mV)'
# title                 = 'NNC_RepeatedKFold_prediction_uncertainty_eachFold_' + KW_name
# vmin1, vmax1          = 0,  900
# vmin2, vmax2          = 50, 150
# plot_prediction_uncertainty(coord_x, coord_y, index_PVD_x_y, C2_new_pred_mean, C2_new_pred_std, 
#                             pred_label, unc_label, title, vmin1, vmax1, vmin2, vmax2, model_path_bo)

# # NNH_NNC_AVG predictions
# title = 'NNH_NNC_RepeatedKFold_prediction_uncertainty_AVG_' + KW_name
# plot_prediction_uncertainty_AVG(coord_x, coord_y, index_PVD_x_y, H1_new_pred_stack, C2_new_pred_stack, 
#                                 title, KW_name, model_path_bo)
